In [1]:
from neat2048.game import Game2048, UP, DOWN, LEFT, RIGHT


def fill_board(game):
    for i in range(4):
        for j in range(4):
            if i < 3 and j < 3:
                game[i][j] = 2

In [ ]:
BOARD_SIZE_X = 4
BOARD_SIZE_Y = 4


def calc_board_monotonicity(game: Game2048) -> int:
    monotonicity = 0

    for x in range(0, BOARD_SIZE_X):
        right_to_left_count = 0
        left_to_right_count = 0

        for y in range(0, BOARD_SIZE_Y - 1):
            if game[x][y] > game[x][y + 1]:
                right_to_left_count += 1
            elif game[x][y] < game[x][y + 1]:
                left_to_right_count += 1

        monotonicity += max(right_to_left_count, left_to_right_count)

    for y in range(0, BOARD_SIZE_Y):
        top_to_bottom_count = 0
        bottom_to_top_count = 0

        for x in range(0, BOARD_SIZE_X - 1):
            if game[x][y] > game[x + 1][y]:
                top_to_bottom_count += 1
            elif game[x][y] < game[x + 1][y]:
                bottom_to_top_count += 1

        monotonicity += max(top_to_bottom_count, bottom_to_top_count)

    return monotonicity

In [2]:
from IPython.display import clear_output

game = Game2048()
game.add_random_tile()

print(repr(game))

while not game.game_end:
    move = input("Enter move: ")

    # clear output
    clear_output(wait=True)

    if move == "q":
        break
    elif move == "w":
        game.move(UP, add_random_tile=True)
    elif move == "s":
        game.move(DOWN, add_random_tile=True)
    elif move == "a":
        game.move(LEFT, add_random_tile=True)
    elif move == "d":
        game.move(RIGHT, add_random_tile=True)
    else:
        print("Invalid move")

    print("Score: ", game.score)
    print(repr(game))

Invalid move
Score:  88
         
 0 2 8 8 
         
 2 2 4 8 
         
 0 0 4 8 
         
 0 0 0 8 
         



In [ ]:
import itertools


def process_long_falling_from_tile(game: Game2048, x: int, y: int) -> int:
    # Min value is 2, max is 4*4 = 16

    best_neighbour_positions = []
    best_neighbour_value = -1

    is_valid = lambda x, y: 0 <= x < game.size_x and 0 <= y < game.size_y

    for dx, dy in [(1, 0), (0, 1), (-1, 0), (0, -1)]:
        new_x, new_y = x + dx, y + dy

        if not is_valid(new_x, new_y):
            continue

        value = game.board[new_x][new_y]
        if value > best_neighbour_value and value < game.board[x][y]:
            best_neighbour_value = value
            best_neighbour_positions = [(new_x, new_y)]
        elif value == best_neighbour_value:
            best_neighbour_positions.append((new_x, new_y))

    best_val = 1
    for x, y in best_neighbour_positions:
        best_val = max(best_val, 1 + process_long_falling_from_tile(game, x, y))

    return best_val


def find_max_path_from_tiles(game: Game2048) -> int:
    # Min value is 2, max is 4*4 = 16
    max_tile_value = 0
    for x, y in itertools.product(range(game.size_x), range(game.size_y)):
        if game.board[x][y] > max_tile_value:
            max_tile_value = game.board[x][y]

    longest_path = 0
    for x, y in itertools.product(range(game.size_x), range(game.size_y)):
        if game.board[x][y] == max_tile_value:
            longest_path = max(longest_path, process_long_falling_from_tile(game, x, y))

    return longest_path

In [ ]:
find_max_path_from_tiles(game)

In [ ]:
game = Game2048()

IDEAL_MONOTONICITY_BOARD = [
    [2, 4, 8, 16],
    [256, 128, 64, 32],
    [512, 1024, 2048, 4096],
    [65536, 32768, 16384, 8192],
]

for y in range(4):
    for x in range(4):
        game[y][x] = IDEAL_MONOTONICITY_BOARD[y][x]

print(find_max_path_from_tiles(game))